In [1]:
from story_sage.story_sage import StorySage
import yaml
import json

In [2]:
import logging
import sys

config_path = './config.yml'


try:
    # Load configuration and data files
    with open(config_path, 'r') as file:
        config = yaml.safe_load(file)
    with open(config['SERIES_PATH'], 'r') as file:
        series_list = yaml.safe_load(file)
    with open(config['ENTITIES_PATH'], 'r') as file:
        entities = json.load(file)
except Exception as e:
    raise e

# Extract configuration settings
api_key = config['OPENAI_API_KEY']
chroma_path = config['CHROMA_PATH']
chroma_collection = config['CHROMA_COLLECTION']

story_sage = StorySage(
    api_key=api_key,
    chroma_path=chroma_path,
    chroma_collection_name=chroma_collection,
    entities=entities,
    series=series_list,
    n_chunks=10  # Number of text chunks to process
)

Sample Questions with Explanations:

Factual Recall:

Q: "What is Harry's aunt and uncle's address?"

Why: Simple fact that should be in early chapters. Tests basic retrieval accuracy.

Character Understanding:

Q: "How does Hermione's attitude toward house rules change from when we first meet her to [current chapter]?"

Why: Tests ability to track character development across multiple chunks and synthesize information.

Plot Comprehension:

Q: "What events led to Harry joining the Quidditch team?"

Why: Tests ability to connect multiple related events and maintain correct sequence.

Spoiler Awareness:

Q: "What do we know about Professor Snape's motivations so far?" (at various points in the series)

Why: Tests system's ability to provide appropriate information based on reading progress.

Complex Reasoning:

Q: "Based on what we've seen so far, why might the Sorting Hat have considered putting Harry in Slytherin?"

Why: Tests ability to synthesize character traits, events, and subtle hints.

Edge Cases:

Q: "What color was the scarf Madam Malkin was wearing when Harry first visited her shop?"

Why: Tests retrieval of minor details and embedding sensitivity.

Multi-Book Progressive:

Q: "How has Harry's relationship with the Dursleys evolved up to this point?"

Why: Tests ability to track developments across multiple books while respecting progress limits.

Contextual Boundaries:

Q: "What do we know about the Chamber of Secrets?" (asked at different points in Book 2)

Why: Tests ability to provide appropriate information based on specific chapter progress.

Character Relationships:

Q: "Describe the friendship between Harry, Ron, and Hermione up to this point."

Why: Tests ability to synthesize relationship dynamics across multiple scenes and chapters.

World-Building Details:

Q: "What have we learned about how the Hogwarts house point system works?"

Why: Tests ability to collect and synthesize scattered world-building details.

For evaluation purposes, I recommend:

- Ask each question at different reading progress points
- Create variations of similar questions to test consistency
- Compare responses across different models
- Test with different chunk sizes
- Evaluate with different embedding techniques
- Try different prompt engineering approaches

In [7]:
QUESTION_LIST = [
    "What is Harry's aunt and uncle's address?",
    "How does Hermione's attitude toward house rules change from when we first meet her to now?",
    "What events led to Harry joining the Quidditch team?",
    "What do we know about Professor Snape's motivations so far?",
    "Based on what we've seen so far, why might the Sorting Hat have considered putting Harry in Slytherin?",
    "What color was the scarf Madam Malkin was wearing when Harry first visited her shop?",
    "How has Harry's relationship with the Dursleys evolved up to this point?",
    "What do we know about the Chamber of Secrets?",
    "Describe the friendship between Harry, Ron, and Hermione up to this point.",
    "What have we learned about how the Hogwarts house point system works?"
]

series_id = '2'

def get_answer(question, book_number, chapter_number):
    data = {
        'question': question,
        'book_number': book_number, 
        'chapter_number': chapter_number,
        'series_id': series_id
    }
    result, context, request_id = story_sage.invoke(**data)
    return (question, result)


In [8]:
results = {'Book 2': [], 'Book 7': []}

for question in QUESTION_LIST:
    result = get_answer(question, 2, 99)
    results['Book 2'].append(result)
    result = get_answer(question, 7, 99)
    results['Book 7'].append(result)

2024-12-24 08:32:25,755 - story_sage.story_sage - INFO - Processing question: What is Harry's aunt and uncle's address? [Request ID: 24519166-c7fe-4fcb-8229-f3efa5791388]
2024-12-24 08:32:31,514 - story_sage.story_sage - INFO - Processing question: What is Harry's aunt and uncle's address? [Request ID: 32c2b81d-4f0b-470d-88f0-da0837127445]
2024-12-24 08:32:36,627 - story_sage.story_sage - INFO - Processing question: How does Hermione's attitude toward house rules change from when we first meet her to now? [Request ID: 1714b9a9-4302-4378-929f-0cc631b892e8]
2024-12-24 08:32:43,789 - story_sage.story_sage - INFO - Processing question: How does Hermione's attitude toward house rules change from when we first meet her to now? [Request ID: 97a03e7d-359d-4059-ae03-30be34bb92bc]
2024-12-24 08:32:54,692 - story_sage.story_sage - INFO - Processing question: What events led to Harry joining the Quidditch team? [Request ID: 4080a60b-3224-4644-b8a8-7ed71f073a1c]
2024-12-24 08:33:03,245 - story_sage

In [23]:
from IPython.display import display, HTML
import markdown

for book, data in results.items():
    html_table = f'<h3>Results for book {book}:</h3><table><tr><th style="text-align: left;">Question</th><th style="text-align: left;">Answer</th></tr>'
    for entry in data:
        html_table += f'<tr><td style="text-align: left;">{entry[0]}</td><td style="text-align: left;">{markdown.markdown(entry[1])}</td></tr>'
    html_table += "</table>"
    styled_table = f'<div style="max-width: 1200px; overflow-x: auto; font-size: 14px;">{html_table}</div>'
    display(HTML(styled_table))

Question,Answer
What is Harry's aunt and uncle's address?,I don't have enough information to answer the question about Harry's aunt and uncle's address.
How does Hermione's attitude toward house rules change from when we first meet her to now?,"In Book 2, Chapter 9, Hermione's attitude initially reflects a strict adherence to school rules when she expresses concern about breaking them: “We’d be breaking about fifty school rules, I expect” (Book 2, Chapter 9). This shows her cautious nature and her tendency to prioritize rules. However, as the chapter progresses, Hermione becomes more willing to engage in rule-breaking when she suggests using Polyjuice Potion to gather information: “No, it’s not... All we’d need would be some Polyjuice Potion” (Book 2, Chapter 9). This indicates a shift in her attitude toward rules, as she is now considering breaking them for a greater purpose. By the end of the same chapter, despite her initial reluctance, Hermione is shown to be fully immersed in their quest and is even driven to find solutions, such as wanting to read about the Chamber of Secrets: “All the copies of Hogwarts, A History have been taken out” (Book 2, Chapter 9). In earlier chapters, Hermione often disapproves of reckless behavior or rule-breaking, as seen when she reacts to Peeves' teasing of Moaning Myrtle: “Oh, dear,” said Hermione sadly (Book 2, Chapter 8). This shows her empathetic side and a desire to uphold a certain moral standard. Overall, Hermione's attitude toward house rules has evolved from strict adherence to a more flexible approach, where she is willing to break rules if she believes it is necessary for the greater good."
What events led to Harry joining the Quidditch team?,"Harry joined the Quidditch team due to a series of events involving a stolen Remembrall and his exceptional flying skills. Here are the key points that led to his joining: Stolen Remembrall Incident: Draco Malfoy stole Neville's Remembrall, prompting Harry to chase after him on his broom. This incident caught the attention of Oliver Wood, the Gryffindor Quidditch captain. (Book 1, Chapter 10) Wood's Impressed Reaction: After witnessing Harry's flying abilities during the chase, Oliver Wood was impressed and decided to recruit Harry as the Seeker for the Gryffindor team. He later expresses confidence in Harry's potential to excel in Quidditch. (Book 1, Chapter 10) Receiving the Nimbus Two Thousand: Following this incident, Harry received a Nimbus Two Thousand broomstick, which further solidified his position on the team. Wood mentions that Harry's joining the team was a result of the special circumstances surrounding the Remembrall incident. (Book 1, Chapter 10) Training and Preparation: Harry underwent several training sessions with Wood, who was keen on developing Harry's skills as a Seeker. Harry's natural talent for flying became evident as he caught golf balls thrown by Wood during practice. (Book 1, Chapter 10) These events culminated in Harry's participation in his first Quidditch match, where he was set to play against Slytherin. (Book 1, Chapter 11)"
What do we know about Professor Snape's motivations so far?,"In Book 2, Chapter 7, it is suggested that Professor Snape has a close connection with Draco Malfoy, as he gives the Slytherin Quidditch team permission to practice, which may indicate his favoritism towards Slytherin and Malfoy specifically: “I’ve got a specially signed note here from Professor Snape. ‘I, Professor S. Snape, give the Slytherin team permission to practice today on the Quidditch field...’” Snape's motivations seem to revolve around maintaining power and control within Hogwarts. In Chapter 9, when Filch reveals that Snape knows about him being a Squib, Snape's presence in the shadows increases Harry’s sense of foreboding, suggesting that Snape often operates from the shadows, manipulating situations to his advantage: “If I might speak, Headmaster,” said Snape from the shadows, and Harry’s sense of foreboding incr

Question,Answer
What is Harry's aunt and uncle's address?,I don't have enough information to answer the question about Harry's aunt and uncle's address.
How does Hermione's attitude toward house rules change from when we first meet her to now?,"Hermione's attitude toward house rules shows significant development from when we first meet her to the current events in Book 7. Initially, she is portrayed as a rule-follower who values structure and safety, but as the series progresses, particularly in Book 7, her attitude shifts towards a more flexible and determined stance in the face of danger. Initial Attitude: Hermione is often seen advocating for rules and safety. In Book 7, Chapter 6, when discussing the dangers of their mission, her protective nature is evident as she insists on understanding the risks involved: “I don’t see that you have to go either!” (Chapter 6). This shows her inclination to adhere to the safety rules and norms set by adults, such as Mrs. Weasley. Change in Attitude: By Book 7, Chapter 22, her attitude evolves as she becomes more assertive about her decisions. She expresses her commitment to their mission despite the risks: “We’re coming with you. That was decided months ago — years, really.” This signifies her departure from strictly following house rules and her determination to face danger alongside her friends. Increased Resolve: In Chapter 9, her initial resistance to splitting up shows her protective instincts, but she also acknowledges the necessity of their decisions: “Hermione firmly... ‘We need a safe place to hide.’” This indicates that while she values safety, she is willing to compromise her previous attitudes towards house rules for the greater good. Overall, Hermione transitions from a character who primarily upholds rules to one who recognizes when to bend them for survival and loyalty to her friends."
What events led to Harry joining the Quidditch team?,"Harry joined the Quidditch team due to a combination of talent and circumstances. Here are the key events that led to his joining: Discovery of Talent: In Book 1, Chapter 10, Oliver Wood, the Gryffindor Quidditch captain, recognizes Harry's potential as a Seeker during a practice session. After throwing golf balls for Harry to catch, Wood is delighted when Harry doesn't miss a single one: ""Wood was delighted"" (Book 1, Chapter 10). Secrecy and Surprise: Wood decides to keep Harry's position as Seeker a secret, referring to him as their ""secret weapon."" The news leaks out, creating a buzz among students, which adds pressure on Harry: ""Hardly anyone had seen Harry play because Wood had decided that, as their secret weapon, Harry should be kept, well, secret"" (Book 1, Chapter 11). First Match Anticipation: As the Quidditch season begins, Harry prepares for his first match against Slytherin, which heightens his anticipation and anxiety about playing: ""On Saturday, Harry would be playing in his first match after weeks of training: Gryffindor versus Slytherin"" (Book 1, Chapter 11). These events highlight how Harry's natural talent, combined with Wood's belief in him as a player, led to his exciting yet nerve-wracking entry into the world of Quidditch at Hogwarts."
What do we know about Professor Snape's motivations so far?,"Professor Snape's motivations throughout the series are complex and multifaceted. Here are some key points derived from the provided context: Loyalty to Dumbledore: Snape's actions often reflect his loyalty to Dumbledore and his plans, even when they conflict with his own feelings. For instance, in Book 7, Chapter 33, Dumbledore acknowledges Snape's agreement to keep an eye on Harry Potter, indicating that Snape's motivations are tied to fulfilling Dumbledore's wishes: “You gave me your word, Severus. … Harry must not know, not until the last moment, not until it is necessary...” (Book 7, Chapter 33). Protective Instincts: Snape's motivations are deeply influenced by his past and his protective feelings, particularly 